<a href="https://colab.research.google.com/github/kropotin4/Notebooks/blob/master/exStar1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install kora -q
from kora import drive
drive.link_nbs()

# Import NeuralNet from another notebook 
from NeuralNet import FFSNNetwork

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import r2_score

In [43]:
#@title ## Параметры Star1 

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/data/" #@param {type:"string"}
 
N_VAR = 6       #@param {type:"number"}
TRAIN_PR = 0.8  #@param {type:"slider", min:0, max:1, step:0.05}

In [44]:
dtype = np.double

In [45]:
path_end = str(N_VAR).zfill(4) + ".csv"
train_features = pd.read_csv(DATA_PATH + "lab1/train_features_" + path_end, header=None, squeeze=True, dtype=dtype)
train_labels   = pd.read_csv(DATA_PATH + "lab1/train_labels_"   + path_end, header=None, squeeze=False, dtype=dtype)
test_features  = pd.read_csv(DATA_PATH + "lab1/test_features_"  + path_end, header=None, squeeze=True, dtype=dtype)

test_labels    = pd.read_csv(DATA_PATH + "lab1.csv", header=None, squeeze=True, dtype=dtype)

hack_features = pd.read_csv(DATA_PATH + "hack_features.csv", header=None, squeeze=True, dtype=dtype)
hack_labels   = pd.read_csv(DATA_PATH + "hack_labels.csv", header=None, squeeze=False, dtype=dtype)

In [46]:
hack_features = pd.DataFrame({
    "x1": hack_features
})

train_features = pd.DataFrame({
    "x1": train_features
    })

In [55]:
# Numpy
 
train_features_np = train_features.to_numpy() 
train_labels_np = train_labels.to_numpy()
 
test_features_np = test_features.to_numpy()

hack_features_np = hack_features.to_numpy()
hack_labels_np = hack_labels.to_numpy()

In [49]:
#@title Тренировка сети: параметры
 
learning_rate = 0.000001 #@param {type:"number"}
epochs =  6000#@param {type:"integer"}
score = 'mse' #@param ["mse", "log"]
calc_type = 'cpu' #@param ["cpu", "gpu"]
#@markdown ---
display_loss=False #@param {type:"boolean"}
initialise=True #@param {type:"boolean"}
#@markdown ---
adv_train = False #@param {type:"boolean"}
adv_epochs = 1000 #@param {type:"integer"}

In [50]:
from IPython.display import clear_output

def fit_nn(NNet: FFSNNetwork, 
           n_rounds: int,
           target=None, 
           verbose=True):
    success = 0
    wb_list = []

    for t in range(n_rounds):
        if not verbose:
            print(str(t) + "/" + str(n_rounds))
            
        try:   # hack_features_np, hack_labels_np
            NNet.fit(train_features_np, train_labels_np, \
                learning_rate=learning_rate, epochs=epochs, \
                display_loss=display_loss, calc_type=calc_type, \
                adv_train=adv_train, adv_epochs=adv_epochs, score=score, initialise=initialise)
            
        except Exception as e:
            if verbose:
                print(str(t) + ": fail; " + str(e))
        else:
            success += 1
            r2 = r2_score(hack_labels.to_numpy(), NNet.predict(hack_features.to_numpy()))
            wb_list.append({'score': r2, 'wb': NNet.get_wb()})
            if verbose:
                print(str(t) + ": success (" + str(r2) + ")")
            if (target != None):
                if (r2 >= target):
                    break

        finally:
            if not verbose:
                clear_output(wait=True)
    
    print("Succes ratio: " + str(success / n_rounds))
    return wb_list

In [51]:
import math
m = 1
n_object = len(train_features_np)
hid_nodes1 = math.sqrt((m + 2)*n_object) + 2*math.sqrt(n_object / (m + 2))
hid_nodes2 = m*math.sqrt(n_object / (m + 2))

print("First layer:" + str(math.ceil(hid_nodes1)))
print("Second layer:" + str(math.ceil(hid_nodes2)))

First layer:25
Second layer:5


In [52]:
hidden_sizes = [1,19,19]
NNet = FFSNNetwork(1, 1, hidden_sizes=hidden_sizes)

In [53]:
#NNet.read_weight_bias(DATA_PATH + 'FFSNN_wb.pcl')

In [ ]:
wb_list = fit_nn(NNet, n_rounds=50, target=0.95, verbose=False)

In [ ]:
len_b = len(wb_list)
wb_list = [item for item in wb_list if item['score'] >= 0.8]
len_a = len(wb_list)
print("wb_list: " + str(len_b) + "->" + str(len_a))

In [ ]:
wb_list.sort(reverse=True, key=lambda d: d['score'])
wb_plot = [d['score'] for d in wb_list]
plt.plot(wb_plot)

In [ ]:
wb_item = wb_list[0]
NNet.set_wb(w=wb_item['wb']['w'], b=wb_item['wb']['b'])

NNet.write_weight_bias(DATA_PATH + 'FFSNN_wb.pcl')

In [ ]:
#NNet.read_weight_bias(DATA_PATH + 'FFSNN_wb_hard.pcl')

In [ ]:
#NNet.set_wb(w=wb_item['wb']['w'], b=wb_item['wb']['b'])

# hack_features.to_numpy(), hack_labels.to_numpy()
# train_features_np, train_labels_np
NNet.fit(hack_features.to_numpy(), hack_labels.to_numpy(), \
            learning_rate=learning_rate, epochs=10000, \
            display_loss=display_loss, calc_type=calc_type, \
            adv_train=adv_train, adv_epochs=adv_epochs, score=score, initialise=False)



In [ ]:
#NNet.write_weight_bias(DATA_PATH + 'FFSNN_wb_hard.pcl')

In [ ]:
NNet_pred = NNet.predict(train_features_np)
NNet_pred2 = NNet.predict(test_features_np)
NNet_pred3 = NNet.predict(hack_features.to_numpy())
print(r2_score(train_labels_np, NNet_pred))
print(r2_score(test_labels.to_numpy(), NNet_pred2))
print(r2_score(hack_labels.to_numpy(), NNet_pred3))

In [ ]:
#pd.Series(NNet_pred2).to_csv(DATA_PATH + "lab1_star.csv", header=None, index=False)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(train_features_np, NNet_pred, c='r', s=5)
ax.scatter(train_features_np, train_labels_np, c='g', s=5)
fig.suptitle("Train data")
 
fig2, ax2 = plt.subplots()
ax2.scatter(test_features_np, NNet_pred2, c='r', s=5)
ax2.scatter(test_features_np, test_labels.to_numpy(), c='b', s=5)
fig2.suptitle("Test data")

fig3, ax3 = plt.subplots()
ax3.scatter(hack_features.to_numpy(), NNet_pred3, c='r', s=5)
ax3.scatter(hack_features.to_numpy(), hack_labels.to_numpy(), c='b', s=5)
fig3.suptitle("Hack data")

plt.show()